# Week 4. Day 1. Exercises from Chapter 10 of FSStDS. 
## Fundamentals of Social Data Science. MT 2022

Within your study pod discuss the following questions. Please submit an individual assignment by 12:30pm Tuesday, November 1st, 2022 on Canvas. 

# Refactoring code 

Chapter 10 gave the example of the Movie Stack Exchange as a file with posts that could be cleaned and imported into Python. The steps taken are sequential and result in a final DataFrame which was pickled. 

Below we want you to proceed in steps, refactor or rewriting that code until we end up with a script whereby you can take the zipped 

# Step 1. 
**Be able to get from the 7z file to the preferred XML file**

In the first step (for which I've provided starter code), you should be able to open a downloaded 7z file representing the archive, export it to a folder under data dir.

## Challenge 1. 
Can you do this with data from the web instead of downloading it first?  

In [8]:
# Answer step 1 below here

# Note you will likely need to install py7zr through pip 
# or use an altnernate approach to unpacking such as 
# pyunpack or libarchive (both of which I found fussy)

In [4]:
from pathlib import Path
from py7zr import unpack_7zarchive
import shutil
import requests


stack = "sustainability"
file_name = f"{stack}.stackexchange.com.7z"
HTML_PATH = f"https://archive.org/download/stackexchange/{file_name}"

DATA_DIR = Path().cwd().parent / "data"
export_path = DATA_DIR / f"{stack}"

if not export_path.exists(): 
    export_path.mkdir()

r = requests.get(HTML_PATH, stream=True)
with open(DATA_DIR / file_name, 'wb') as f:
    f.write(r.content)

In [6]:
try:
    shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
except shutil.RegistryError:
    pass
shutil.unpack_archive(DATA_DIR / file_name, export_path)

# Step 2. 

Refactor the code from Chapter 10 of the book into a function that works for the `posts.xml`. That function should take in the base data and then:
1. Convert the `int` variables (except where they start or end in `Id`) into integer values.(CHECK)
2. Convert the `datetime` variables. (CHECK) 
3. Convert tags data from `str` into a `list`. (CHECK)
4. Create a separate column for `CleanBody` which is the `Body` without HTML. (CHECK) 
5. Assign the HTML into a column as a list called `ListURL`. (CHECK)
6. Pickle that DataFrame with a coherent name, such as `f"{stack}_Posts_cleaned.xml"`.

Notes: 
> I say 'in a function', but you might want to have a main function and then helper functions for subprocesses. 
> You can make this more abstract, but that's coming anyway. Read the exercises below, and then think about this plan.

In [93]:
# Answer Step 2 below here 
import xmltodict
import pandas as pd
import functools
# import beautifulsoup4
import bs4
from typing import List, Optional

def read_xml_rows(file_path: Path) -> dict:
    data_type = file_path.stem.lower()
    with open(file_path, "rb") as f:
        xml = f.read()
    xml = xmltodict.parse(xml)
    rows = xml[data_type]["row"]
    return rows

def read_xml_rows_to_df(file_path: Path) -> pd.DataFrame:
    rows = read_xml_rows(file_path)
    return pd.DataFrame(rows)

def id_cols_to_str(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if col.endswith("Id"):
            df[col] = df[col].astype(str)
    return df


def convert_numeric_to_int(df: pd.DataFrame) -> pd.DataFrame:
    """Check if column contains numbers and convert to int"""
    for col in df.columns:
        if not col.endswith("Id"):
            try:
                df[col] = df[col].astype(int)
            except ValueError:
                pass
    return df

def convert_date_cols(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if col.endswith("Date"):
            df[col] = pd.to_datetime(df[col])
    return df

def tags_to_list(tags: Optional[str]) -> List[str]:
    "turn <tag1><tag2>... into [tag1, tag2, ...]"
    if not isinstance(tags, str):
        return []
    return tags.replace("><", ",").replace("<", "").replace(">", "").split(",")

def convert_tags(df: pd.DataFrame, tag_col: str="@Tags") -> pd.DataFrame:
    df[tag_col] = df[tag_col].apply(tags_to_list)
    return df

def get_url_from_html(html: str) -> List[str]:
    soup = bs4.BeautifulSoup(html, "html.parser")
    return [a["href"] for a in soup.find_all("a", href=True)]

def create_url_col(df: pd.DataFrame) -> pd.DataFrame:
    df["@ListURLs"] = df["@Body"].apply(get_url_from_html)
    return df

def strip_html(text: str) -> str:
    return bs4.BeautifulSoup(text, "html.parser").get_text()

def strip_html_df(df: pd.DataFrame, text_col: str="@Body") -> pd.DataFrame:
    df["@CleanBody"] = df[text_col].apply(strip_html)
    return df

def remove_adds_from_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Remove '@' from the column names"""
    df.columns = [col.replace("@", "") for col in df.columns]
    return df

def pickle_df(df: pd.DataFrame, file_path: Path) -> None:
    df.to_pickle(file_path)


def compose(*funcs):
    def inner(arg):
        return functools.reduce(lambda x, f: f(x), funcs, arg)
    return inner

In [76]:

def main():
    pipeline = compose(
        read_xml_rows_to_df, 
        id_cols_to_str, 
        convert_numeric_to_int, 
        convert_date_cols, 
        convert_tags, 
        strip_html_df, 
        create_url_col,
        remove_adds_from_cols,
    )
    posts = pipeline(DATA_DIR / f"{stack}/Posts.xml")
    # save to pickle
    pickle_df(posts, DATA_DIR / f"{stack}_Posts_cleaned.pkl")

main()

c:\Users\jhr\Anaconda3\envs\sdspython\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\jhr\Anaconda3\envs\sdspython\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


# Step 3. 

Parameterise the function. Depending on how you created the function above, you might have hard coded the names of the columns from the `Posts.xml` data. This time, make a parameter for the specific schema that you are going to use to convert the data. The schema should be stored as JSON and have the type of XML file as a key, with the value being a dictionary for the column names and the conversion, such as the following: 

~~~ Python
["Posts": {"Id":none,
           "PostTypeID":none,...,
           "Tags":["str","list"],
           "AnswerCount":"int",
           "Body":["str","CleanHTML","listHTML"]},
 "Users": {"Id":none,
           "Reputation":"int", ...}
~~~

So, now this time, the main function should read in the Schema from file, select the right table type (such as "Posts") and then return (or export to pickle) a DataFrame with the same naming conventions as above.   

Full schema available here: https://i.stack.imgur.com/AyIkW.png

> Note that this schema or question does not make assumptions about what you will do with None values, but it is encouraged for you to consider whether to convert to None, "", pd.NA, np.NAN, depending on context. 

In [99]:
# Answer Step 3 below here 
import json

def read_json(file_path: Path) -> dict:
    with open(file_path, "r") as f:
        return json.load(f)
    
schema_file = read_json(Path("../stackexchange_schema.json"))

def convert_int_cols(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    int_cols = [col for col, dtype in schema.items() if dtype == "int"]
    for col in int_cols:
        df[col] = df[col].astype(int, errors="ignore")
    return df

def convert_string_cols(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    str_cols = [col for col, dtype in schema.items() if dtype == "str"]
    for col in str_cols:
        df[col] = df[col].astype(str)
    return df

def convert_date_cols(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    date_cols = [col for col, dtype in schema.items() if dtype == "datetime"]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    return df

def choose_cols(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    return df[schema]

def add_taglist(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    if not "Tags" in schema:
        return df
    elif "TagList" not in schema["Tags"]:
        return df
    df["TagList"] = df["Tags"].apply(tags_to_list)
    return df

def clean_html(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    if not "Body" in schema:
        return df
    elif "CleanHTML" not in schema["Body"]:
        return df
    df["CleanHTML"] = df["Body"].apply(strip_html)
    return df

def add_url_list(df: pd.DataFrame, schema: dict) -> pd.DataFrame:
    if not "Body" in schema:
        return df
    elif "ListURL" not in schema["Body"]:
        return df
    df["ListURL"] = df["Body"].apply(get_url_from_html)
    return df

def process_file(file_path: Path) -> None:
    data_type = file_path.stem
    schema = schema_file[data_type]
    
    basic_pipeline = compose(
        read_xml_rows_to_df,
        remove_adds_from_cols,
        functools.partial(choose_cols, schema=schema),
        functools.partial(convert_int_cols, schema=schema),
        functools.partial(convert_string_cols, schema=schema),
        functools.partial(convert_date_cols, schema=schema),
    )
    post_pipeline = compose(
        basic_pipeline,
        functools.partial(add_taglist, schema=schema),
        functools.partial(clean_html, schema=schema),
        functools.partial(add_url_list, schema=schema),
    )

    if data_type == "Posts":
        df = post_pipeline(file_path)
    elif data_type == "Users":
        df = basic_pipeline(file_path)
    else:
        raise ValueError(f"Unknown data type: {data_type}")
    pickle_df(df, DATA_DIR / f"{stack}_{data_type}_cleaned2.pkl")

process_file(DATA_DIR / f"{stack}/Users.xml")
df = pd.read_pickle(DATA_DIR / f"{stack}_Users_cleaned2.pkl")


C:\Users\jhr\AppData\Local\Temp\ipykernel_1260\1162537027.py:29: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  return df[schema]
C:\Users\jhr\AppData\Local\Temp\ipykernel_1260\1162537027.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(int, errors="ignore")
C:\Users\jhr\AppData\Local\Temp\ipykernel_1260\1162537027.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str)


# Step 4. A wide parameter. 

Create a means (either with a parameter in the original file or using a separate file) in order to create a long table from one of the wide tables. That is, it should take the column that is used for the long data (which we assume would be a list of values) and then `explode()` the table. By default, it should only explode the selected column. You should be able to pass it a list of additional columns that will also appear in the exploded data.  

In [ ]:
# Answer Step 4 below here


# Challenge #2. 
_(example code will not be provided)_

Recall that we downloaded the data from the Internet Archive. That main URL has a list of all of the Stack Exchanges and their sizes. 

Can we use this data in order to present a list of Stack Exchanges and then have the user select which one to first download instead of linking directly? 

Explore this as well as some packages for providing progress bars for the download process. Package all the code up in a script that allows the user to select which Exchange and then receive the resulting preferred tables as .pkl.

In [ ]:
# Try Challenge 2 below here 

